In [21]:
import numpy as np
import pandas as pd
import random
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Sample user data
users = [
    {"id": 1, "name": "Alice", "bio": "Anxiety issues, seeking mindfulness techniques."},
    {"id": 2, "name": "Bob", "bio": "Recovered from depression, wants to help others."},
    {"id": 3, "name": "Charlie", "bio": "Struggles with OCD, looking for peer discussions."},
    {"id": 4, "name": "David", "bio": "Mild depression, exploring therapy options."},
    {"id": 5, "name": "Eve", "bio": "General stress and workplace anxiety."}
]

# Create DataFrame
df = pd.DataFrame(users)

# Text vectorization
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(df['bio'])

# Similarity matrix
similarity_matrix = cosine_similarity(X)

# Clustering
num_clusters = 2
kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
df['cluster'] = kmeans.fit_predict(X)

# Peer Match Finder
def find_peer(user_id):
    user_index = df[df['id'] == user_id].index[0]
    cluster_id = df.loc[user_index, 'cluster']
    peers = df[(df['cluster'] == cluster_id) & (df['id'] != user_id)]
    peers = peers.copy()  # Avoid SettingWithCopyWarning
    peers['match_score'] = similarity_matrix[user_index][peers.index]
    return peers[['name', 'bio', 'match_score']].sort_values(by='match_score', ascending=False)

# Relaxation Techniques for each problem
relaxation_techniques = {
    "anxiety": (
        "Anxiety can feel overwhelming, but there are ways to manage it:\n"
        "- Try deep breathing: inhale for 4 seconds, hold for 4, exhale for 6.\n"
        "- Practice grounding: focus on 5 things you can see, 4 you can touch, 3 you can hear...\n"
        "- Consider journaling your thoughts to better understand your triggers.\n"
        "Would you like to try a guided relaxation exercise?"
    ),
    "depression": (
        "I hear you, and I'm really sorry you're feeling this way. You're not alone. Here are some things that might help:\n"
        "- Try to get some sunlight, even a short walk outside.\n"
        "- Do one small task—like brushing your hair or drinking water. Progress is progress.\n"
        "- Avoid isolating yourself too much, even a short chat can help.\n"
        "Would you like a mental health resource or support helpline?"
    ),
    "stress": (
        "Stress can sneak up on us. Here are a few ways to cope right now:\n"
        "- Step away for a 5-minute break and breathe deeply.\n"
        "- Write down what's bothering you—seeing it on paper can help reduce its weight.\n"
        "- Move your body—stretching, walking, or dancing can release tension.\n"
        "- Try saying 'no' to things that overwhelm you—boundaries are healthy.\n"
        "Need help with a calming technique?"
    ),
    "ocd": (
        "Managing OCD is tough, but you're doing your best—and that's enough. Consider these steps:\n"
        "- Acknowledge the thought, but remind yourself: 'This is just OCD talking.'\n"
        "- Delay the compulsion by a few minutes. Increase that delay over time.\n"
        "- Practice mindfulness to ground yourself in the present.\n"
        "Would you like to try some self-help techniques?"
    ),
    "insomnia": (
        "Struggling with sleep is tough. Here are some sleep hygiene tips:\n"
        "- Stick to a consistent bedtime and wake-up time.\n"
        "- Avoid screens 1 hour before bed.\n"
        "- Use white noise or calm music to relax.\n"
        "Want help creating a calming bedtime routine?"
    ),
    "panic": (
        "Panic attacks can be scary, but they pass. Try:\n"
        "- Focus on slow, deep breaths—count 4 in, 4 hold, 6 out.\n"
        "- Splash cold water on your face.\n"
        "- Sit somewhere safe and remind yourself: 'This will pass.'\n"
        "Would you like to walk through a grounding technique?"
    ),
    "loneliness": (
        "Feeling lonely is tough, but you’re not alone. Consider:\n"
        "- Joining a community or hobby group (online or local).\n"
        "- Volunteering or helping others—it builds connection.\n"
        "- Reaching out to old friends or peers.\n"
        "Would you like to be matched with someone to talk to?"
    ),
    "burnout": (
        "Burnout can leave you emotionally drained. Try these recovery steps:\n"
        "- Take time off—even a short walk or break.\n"
        "- Practice saying 'no' to new responsibilities.\n"
        "- Make space for something fun or relaxing.\n"
        "- Prioritize sleep and hydration.\n"
        "Would you like to build a recovery checklist together?"
    )
}

emotion_keywords = {
    "anxiety": ["anxious", "nervous", "panic", "fear", "uneasy", "worry", "apprehensive", "restless"],
    "depression": ["sad", "down", "depressed", "hopeless", "empty", "low", "unmotivated", "isolated"],
    "stress": ["stressed", "tense", "burnout", "pressure", "overwhelmed", "frustrated", "strained", "heavy"],
    "ocd": ["compulsion", "ritual", "repetitive", "ocd", "obsession", "intrusive", "control"],
    "insomnia": ["insomnia", "sleep", "tired", "fatigue", "awake", "restless", "night", "sleepless"],
    "panic": ["panic", "fear", "overwhelmed", "attack", "nervous", "dizzy", "choking", "racing heart"],
    "loneliness": ["lonely", "isolated", "alone", "no one", "disconnect", "emptiness", "withdrawn", "friendless"],
    "burnout": ["burnout", "exhausted", "drained", "fatigued", "overworked", "mentally tired", "overwhelmed", "empty"]
}

def detect_emotion(user_input):
    user_input = user_input.lower()
    for emotion, keywords in emotion_keywords.items():
        if any(word in user_input for word in keywords):
            return emotion
    return None

def chatbot_response(user_input, ongoing_conversation=True):
    emotion = detect_emotion(user_input)
    if emotion and emotion in relaxation_techniques:
        return relaxation_techniques[emotion]
    for key in relaxation_techniques:
        if key in user_input.lower():
            return relaxation_techniques[key]

    # Handle response when asking to "Tell me more"
    if "can you tell me more" in user_input.lower():
        return "I'm really glad you're sharing more with me. Would you like to continue (yes) or end the conversation (no)?"

    # Check if user wants to continue or end the conversation
    if "no" in user_input.lower():
        return "Okay, I understand. If you need anything, feel free to reach out anytime."

    if "yes" in user_input.lower():
        return "Feel free to tell me anything else that's on your mind, or I can suggest a few techniques to help you feel better."

    return "I'm here to listen. Can you tell me more?"

# Register new user
def register_user():
    name = input("Enter your name: ")
    bio = input("Describe what you're going through: ")
    new_id = df['id'].max() + 1
    df.loc[len(df.index)] = [new_id, name, bio, -1]
    return new_id

# Save matches
def export_peers(peers_df, filename='peer_matches.json'):
    peers_df.to_json(filename, orient='records', lines=True)
    print(f"Peer matches saved to {filename}")

# Chat interface
def chat_with_bot(user_id):
    print("\n[Bot]: Hello! How can I support you today? (Type 'bye' to exit)")
    ongoing_conversation = True
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'bye':
            print("[Bot]: Take care! Reach out anytime.")
            break
        response = chatbot_response(user_input, ongoing_conversation)
        print(f"[Bot]: {response}")

        # Exit if the user says "no" after "Can you tell me more?"
        if "no" in user_input.lower() and "can you tell me more" in user_input.lower():
            break

# Entry point
if __name__ == "__main__":
    print("Welcome to the AI-Powered Mental Health Peer Support Community 💬")
    choice = input("Do you want to register as a new user? (yes/no): ").strip().lower()

    if choice == 'yes':
        new_user_id = register_user()
        print(f"Registered! Your user ID is {new_user_id}")
    else:
        new_user_id = int(input("Enter your user ID: "))

    # Recompute clustering for updated data
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(df['bio'])
    similarity_matrix = cosine_similarity(X)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    df['cluster'] = kmeans.fit_predict(X)

    # Start the chat with the bot
    chat_with_bot(new_user_id)


Welcome to the AI-Powered Mental Health Peer Support Community 💬
Do you want to register as a new user? (yes/no): yes
Enter your name: Ayush
Describe what you're going through: I'm feeling really anxious today.
Registered! Your user ID is 6

[Bot]: Hello! How can I support you today? (Type 'bye' to exit)
You: I'm feeling really anxious today.
[Bot]: Anxiety can feel overwhelming, but there are ways to manage it:
- Try deep breathing: inhale for 4 seconds, hold for 4, exhale for 6.
- Practice grounding: focus on 5 things you can see, 4 you can touch, 3 you can hear...
- Consider journaling your thoughts to better understand your triggers.
Would you like to try a guided relaxation exercise?
You: Yes, please.
[Bot]: Feel free to tell me anything else that's on your mind, or I can suggest a few techniques to help you feel better.
You: I'm also having trouble sleeping at night.
[Bot]: Struggling with sleep is tough. Here are some sleep hygiene tips:
- Stick to a consistent bedtime and wake-